# Get down the work flow with Hackathon 1b

In [1]:
#import and magics
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

create a file to load as parameters

In [ ]:
import json
import hackathon_params as hp

hp.hackParams(10,2)   #this method creates a json file with dictionary of parameters {N, steps}

# hackathon file

1) load the file of interest, remove any Viewer stuff from it, 

2) insert the variables from the parameter file so that we can change them, 

3) load the json parameter file from argv argument

4) save the results in some sort of file

In [ ]:
# %load fipy_hackathon_1b.py
import fipy as fp
import numpy as np
import json
import os
import hackathon_params as hp
import sys

#load in the json parameter file here
jsonfile = sys.argv[1]

if jsonfile:
    with open(jsonfile, 'rb') as ff:
        params = json.load(ff)
        
else:
    params = dict()
    
print 'my params:'. params

#extract the parameters
N= params.get('N', 20)
total steps = params.get('steps', 2)
sumatra_label = params.get('sumatra_label', '')

mesh = fp.Grid2D(nx=N, ny=N, dx=0.5, dy=0.5)

c_alpha = 0.05
c_beta = 0.95
A = 2.0
kappa = 2.0
c_m = (c_alpha + c_beta) / 2.
B = A / (c_alpha - c_m)**2
D = D_alpha = D_beta = 2. / (c_beta - c_alpha)
c_0 = 0.45
q = np.sqrt((2., 3.))
epsilon = 0.01

c_var = fp.CellVariable(mesh=mesh, name=r"$c$", hasOld=True)

r = np.array((mesh.x, mesh.y))
c_var[:] = c_0 + epsilon * np.cos((q[:, None] * r).sum(0))

f_0_var = -A + 3*B*(c_var - c_m)**2 + 3*c_alpha*(c_var - c_alpha)**2 + 3*c_beta*(c_var - c_beta)**2

eqn = fp.TransientTerm(coeff=1.) == fp.DiffusionTerm(D * f_0_var) - fp.DiffusionTerm((D, kappa))

elapsed = 0.0
steps = 0
dt = 0.01
total_sweeps = 2
tolerance = 1e-1
# total_steps = 300

c_var[:] = c_0 + epsilon * np.cos((q[:, None] * r).sum(0))

c_var.updateOld()

from fipy.solvers.pysparse import LinearLUSolver as Solver

solver = Solver()

while steps < total_steps:
    res0 = eqn.sweep(c_var, dt=dt, solver=solver)

    for sweeps in range(total_sweeps):
        res = eqn.sweep(c_var, dt=dt, solver=solver)

        print ' '
        print 'steps',steps
        print 'res',res
        print 'sweeps',sweeps
        print 'dt',dt


    if res < res0 * tolerance:
        steps += 1
        elapsed += dt
        dt *= 1.1
        c_var.updateOld()
    else:
        dt *= 0.8
        c_var[:] = c_var.old
        
#save the results of c_var
#create a filepath where to save the data file
filepath = os.path.join('Data', sumatra_label)
filename = 'c_var_outputs_1b.txt'
np.savetxt(os.path.join(filepath, filename), np.array(c))